In [1]:
pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Access to Loan API Endpoint
import requests


In [2]:
# Create a Python program to GET (consume) data from the above API endpoint for the loan application dataset.
url='https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
response = requests.get(url)
data=response.json()


In [3]:
# Find the status code of the above API endpoint.
# Hint: status code could be 200, 400, 404, 401.
#200	OK	The requested action was successful.
response.status_code

200

In [4]:
# Method1: Load data via Pandas Library
import pandas as pd
df = pd.read_json (r'{}'.format(url))
df.head()


,Application_ID,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status
0,LP001002,Male,No,0,Graduate,No,1,Urban,medium,Y
1,LP001003,Male,Yes,1,Graduate,No,1,Rural,medium,N
2,LP001005,Male,Yes,0,Graduate,Yes,1,Urban,low,Y
3,LP001006,Male,Yes,0,Not Graduate,No,1,Urban,low,Y
4,LP001008,Male,No,0,Graduate,No,1,Urban,medium,Y


In [5]:
#Importing the Libraries
import pyspark
from pyspark.sql import SparkSession

# Creating Spark Session
spark = SparkSession.builder.appName('LoanApplication').getOrCreate()


#Method1: continue
#Create PySpark DataFrame from Pandas: Jave Error
#sparkDF=spark.createDataFrame(df) 
#sparkDF.printSchema()
#sparkDF.show()

# Method2: Load data via Pyspark
file = requests.get(url, allow_redirects=True)

open('loan_data.json', 'wb').write(file.content)
#open('loan_data.json', 'w').write(file.text)

sparkdf = spark.read.json('loan_data.json')

sparkdf.printSchema()
sparkdf.show()



root
 |-- Application_ID: string (nullable = true)
 |-- Application_Status: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Self_Employed: string (nullable = true)

+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    G

In [7]:
# Once Python reads data from the API, utilize PySpark to load data into RDBMS(SQL). The table name should be CDW-SAPP_loan_application in the database.

# Note: Use the “creditcard_capstone” database.

sdf=sparkdf.select("Application_ID","Application_Status","Credit_History","Dependents","Education","Gender","Income","Married","Property_Area","Self_Employed")

sdf.write.format("jdbc").mode("overwrite").option("driver","com.mysql.jdbc.Driver")\
    .option("url", "jdbc:mysql://{}:3306/creditcard_capstone".format(os.getenv("MYSQL_HOST"))) \
	.option("dbtable", "CDW_SAPP_loan_application") \
	.option("user", os.getenv("MYSQL_USER")).option("password", os.getenv("MYSQL_PASS")).save()